## Data Preprocesing for making accurate predictions ##

In this project, we aim to predict the price of bitcoin using sentiment analysis from twitter data and finalcial features to support the sentiment, using Random Forrest Regression Model.

In order to meature porality of tweets, where a tweet is positive/optimistc, negative/pessimistic or neutral/realistic, we will use the Natural language processing tool Vader. 


# Disclaimer #
Please be sure to check if you have all the libraries required if not, please run pip install -r requirements.txt(file in repo) in your command line. Since the bitcoin tweets.csv file that will be used has 95762468 data points, some cells might take a while to run. If you'd rather jump to just running the predictions, i have saved
the cleaned and preprocessed dataset in the repository title - Bitcoin_tweets_Price_data.csv, which is the result of the this project. The process for making predictions are in prediction_model.ipynb

In [ ]:
#importing libraries
#!pip install numpy
#!pip install pandas
#!pip install matplotlib
#!pip install plotly
#!pip install vaderSentiment
#!pip install yfinance
#!pip install scikit-learn
#!pip install statsmodels
#!pip install graphviz

#get_ipython().system('pip install clean-text')
#get_ipython().system('pip install -r requirements.txt')

#Data Source
import numpy as np
import pandas as pd
import matplotlib as plt
import re
import yfinance as yf
#Data visualization
import plotly.graph_objs as go

In [2]:
btc_usd= yf.Ticker("BTC-USD")

#Get Bitcoin data
bitdata = btc_usd.history(period = '3y', interval= '1d')

In [3]:
bitdata.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-07-04 00:00:00+00:00,9084.233398,9183.295898,9053.629883,9132.488281,12290528515,0,0
2020-07-05 00:00:00+00:00,9126.090820,9162.183594,8977.015625,9073.942383,12903406143,0,0
2020-07-06 00:00:00+00:00,9072.849609,9375.474609,9058.664062,9375.474609,17889263252,0,0
2020-07-07 00:00:00+00:00,9349.161133,9360.617188,9201.815430,9252.277344,13839652595,0,0
2020-07-08 00:00:00+00:00,9253.020508,9450.335938,9249.500000,9428.333008,19702359883,0,0


In [4]:
bitdata.tail()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-06-30 00:00:00+00:00,30441.353516,31256.863281,29600.275391,30477.251953,26387306197,0,0
2023-07-01 00:00:00+00:00,30471.847656,30641.289062,30328.865234,30590.078125,9086606733,0,0
2023-07-02 00:00:00+00:00,30587.269531,30766.140625,30264.019531,30620.769531,10533418042,0,0
2023-07-03 00:00:00+00:00,30624.515625,31375.613281,30586.513672,31156.439453,15271884873,0,0
2023-07-04 00:00:00+00:00,31140.369141,31325.197266,30911.646484,31015.001953,15794570240,0,0


In [5]:
#convert Date to datetime(as it usually is downloaded in a string type)
bitdata.index = pd.to_datetime(bitdata.index)
bitdata.head(5)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-07-04 00:00:00+00:00,9084.233398,9183.295898,9053.629883,9132.488281,12290528515,0,0
2020-07-05 00:00:00+00:00,9126.090820,9162.183594,8977.015625,9073.942383,12903406143,0,0
2020-07-06 00:00:00+00:00,9072.849609,9375.474609,9058.664062,9375.474609,17889263252,0,0
2020-07-07 00:00:00+00:00,9349.161133,9360.617188,9201.815430,9252.277344,13839652595,0,0
2020-07-08 00:00:00+00:00,9253.020508,9450.335938,9249.500000,9428.333008,19702359883,0,0


In [6]:
#Removes the 0's in time format
bitdata.index =bitdata.index.strftime('%Y-%m-%d')

In [7]:
bitdata.columns
bitdata.head(5)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-07-04,9084.233398,9183.295898,9053.629883,9132.488281,12290528515,0,0
2020-07-05,9126.090820,9162.183594,8977.015625,9073.942383,12903406143,0,0
2020-07-06,9072.849609,9375.474609,9058.664062,9375.474609,17889263252,0,0
2020-07-07,9349.161133,9360.617188,9201.815430,9252.277344,13839652595,0,0
2020-07-08,9253.020508,9450.335938,9249.500000,9428.333008,19702359883,0,0


In [8]:
# create lineplot with bitdata.index as x-axis and bitdata["Close"] as y-axis
#As you can see the highest it has reached was in November 8 2021 when the Closing price was 67,566.83
fig = go.Figure(data=go.Scatter(x=bitdata.index, y=bitdata["Close"]))
fig.show()

In [9]:
#Transaction volume was the highest on February 6th, 2021
fig = go.Figure(data=go.Scatter(x=bitdata.index, y=bitdata["Volume"]))
fig.show()

In [10]:
#Reading bitcoin tweets(huge file dont read it)
bitcoin_tweets = pd.read_csv('Bitcoin_tweets.csv')

C:\Users\arjun\AppData\Local\Temp\ipykernel_20376\2922141733.py:2: DtypeWarning:

Columns (1,2,3,4,5,6,7,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.



In [11]:

bitcoin_tweets.shape

(95762468, 13)

In [12]:
bitcoin_tweets.head(5)

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,DeSota Wilson,"Atlanta, GA","Biz Consultant, real estate, fintech, startups...",2009-04-26 20:05:09,8534.0,7605,4838,False,2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...,['bitcoin'],Twitter Web App,False
1,CryptoND,NaN,😎 BITCOINLIVE is a Dutch platform aimed at inf...,2019-10-17 20:12:10,6769.0,1532,25483,False,2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""...","['Thursday', 'Btc', 'wallet', 'security']",Twitter for Android,False
2,Tdlmatias,"London, England","IM Academy : The best #forex, #SelfEducation, ...",2014-11-10 10:50:37,128.0,332,924,False,2021-02-10 23:54:48,"Guys evening, I have read this article about B...",NaN,Twitter Web App,False
3,Crypto is the future,NaN,I will post a lot of buying signals for BTC tr...,2019-09-28 16:48:12,625.0,129,14,False,2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...,"['Bitcoin', 'FX', 'BTC', 'crypto']",dlvr.it,False
4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,Europa,Co-founder @RENJERJerky | Forbes 30Under30 | I...,2016-02-03 13:15:55,1249.0,1472,10482,False,2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...,['BTC'],Twitter Web App,False


In [ ]:
#removing the timestamp and converting the string datatype of date to datetime format
bitcoin_tweets.date = bitcoin_tweets.date.str[:10] #tweets.date.str[:10]
bitcoin_tweets['date'] = pd.to_datetime(bitcoin_tweets['date'], errors='coerce', format='%Y-%m-%d %H:%M:%S')

In [ ]:
bitcoin_tweets.head(5)

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,DeSota Wilson,"Atlanta, GA","Biz Consultant, real estate, fintech, startups...",2009-04-26 20:05:09,8534.0,7605,4838,False,2021-02-10,Blue Ridge Bank shares halted by NYSE after #b...,['bitcoin'],Twitter Web App,False
1,CryptoND,NaN,😎 BITCOINLIVE is a Dutch platform aimed at inf...,2019-10-17 20:12:10,6769.0,1532,25483,False,2021-02-10,"😎 Today, that's this #Thursday, we will do a ""...","['Thursday', 'Btc', 'wallet', 'security']",Twitter for Android,False
2,Tdlmatias,"London, England","IM Academy : The best #forex, #SelfEducation, ...",2014-11-10 10:50:37,128.0,332,924,False,2021-02-10,"Guys evening, I have read this article about B...",NaN,Twitter Web App,False
3,Crypto is the future,NaN,I will post a lot of buying signals for BTC tr...,2019-09-28 16:48:12,625.0,129,14,False,2021-02-10,$BTC A big chance in a billion! Price: \487264...,"['Bitcoin', 'FX', 'BTC', 'crypto']",dlvr.it,False
4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,Europa,Co-founder @RENJERJerky | Forbes 30Under30 | I...,2016-02-03 13:15:55,1249.0,1472,10482,False,2021-02-10,This network is secured by 9 508 nodes as of t...,['BTC'],Twitter Web App,False


In [ ]:
#High volumes of bitcoin tweets were posted in 2021 so let us only consider all the tweets from 2021 to 2022
bitcoin_tweets_highvol = bitcoin_tweets[bitcoin_tweets['date'].dt.year >= 2021]
#exclude = bitcoin_tweets[bitcoin_tweets['date'].dt.year != 2021]
bitcoin_tweets_highvol.to_csv('bitcoin_tweets_highvol.csv', index=False)


In [ ]:

#Reading bitcoin tweets(the one which has 2021, 2022 tweets)
bitcoin_tweets_highvol = pd.read_csv('bitcoin_tweets_highvol.csv')

C:\Users\arjun\AppData\Local\Temp\ipykernel_33720\1403170844.py:2: DtypeWarning:

Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.



In [ ]:
bitcoin_tweets_highvol.tail(10)

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
4689278,世界杯下注,联系方式↑↑,联系方式：\nTG客服：\nhttps://t.co/dvCBR1DmkM \n\nvx:t...,2017-06-02 15:43:44,1.0,3.0,0.0,False,2023-01-06,Jonas Cumberland #btc #彩票 Merle Noyes #世界杯直播 h...,"['btc', '彩票', '世界杯直播']",Twitter Web App,False
4689279,Gale Browning,NaN,กดซื้อผ่านเว็บ : https://t.co/Wjyr6G2jz5 #ขายเ...,2022-10-29 23:32:12,33.0,120.0,55.0,False,2023-01-06,❤️ Join me at Bybit and earn exclusive rewards...,"['BTC', 'Airdrops', 'USDT', 'BYBIT', 'BANK', '...",Twitter Web App,False
4689280,Jeanie Phillips,NaN,กดซื้อผ่านเว็บ : https://t.co/BBuB3IDZZj #ขายเ...,2022-10-24 12:51:13,22.0,127.0,64.0,False,2023-01-06,❤️ Join me at Bybit and earn exclusive rewards...,"['BTC', 'Airdrops', 'USDT', 'BYBIT', 'BANK', '...",Twitter Web App,False
4689281,世界杯投注平台,联系方式↑↑,联系方式：\nTG客服：\nhttps://t.co/iWzBzNIKo9 \n\nvx:t...,2022-06-25 13:24:21,4.0,16.0,2.0,False,2023-01-06,Lorraine Harvey #btc #彩票 Coral Bart #世界杯直播 htt...,"['btc', '彩票', '世界杯直播']",Twitter Web App,False
4689282,Behzad.moni,NaN,⚙️ The future=dogelonmars - saitama -safemoon ...,2021-11-22 10:26:05,66.0,78.0,1524.0,False,2023-01-06,#DogelonMars is the future. #TSUKA is the nex...,"['DogelonMars', 'TSUKA', 'SHIB', 'SAFEMOON', '...",Twitter Web App,False
4689283,TAnotepad,Not financial advice,#Bitcoin | Forever Alone Crypto Trader 😭 | Ana...,2022-11-04 10:18:11,674.0,1789.0,27466.0,False,2023-01-06,"Bitcoin squeeze is SUPER TIGHT, which way will...","['BTC', 'bitcoin', 'Crypto', 'cryptocurrency',...",Twitter for Android,False
4689284,Boba-Feh,South of the North Pole,My backpack has jets. Reformed Necromonger,2008-04-02 21:48:47,79.0,454.0,125.0,False,2023-01-06,Closed #BTC short at 16725. Missed my long pla...,['BTC'],Twitter for Android,False
4689285,Ethereum Yoda,NaN,UP or DOWN...\n.\n.\n.\n.\nPrice matters NOT.,2022-07-24 04:50:18,532.0,1.0,0.0,False,2023-01-06,#Ethereum price update: \n\n#ETH $1263.59 USD\...,"['Ethereum', 'ETH', 'Bitcoin', 'BTC', 'altcoin...",Twitter Web App,False
4689286,Bitcoin Price Ticker,NaN,Tweets the current price of #bitcoin every 5 m...,2022-10-20 07:10:38,83.0,7.0,9.0,False,2023-01-06,1₿ = $16814.7 -0.07%🔻\n\nDetails:\nChange: 🔻-1...,"['bitcoin', 'btc']",BitcoinLivePriceTicker,False
4689287,faucetojisan,NaN,💰faucetpay 👉\nhttps://t.co/eHqMw9QdFx\n\n💰betf...,2022-10-13 05:41:53,14.0,10.0,0.0,False,2023-01-06,Earn crypto by playing fun games online.\nGet ...,"['faucet', 'cointiply', 'BTC']",Twitter for Android,False


In [ ]:
bitcoin_tweets_highvol.columns

Index(['user_name', 'user_location', 'user_description', 'user_created',
       'user_followers', 'user_friends', 'user_favourites', 'user_verified',
       'date', 'text', 'hashtags', 'source', 'is_retweet'],
      dtype='object')

In [ ]:
bitcoin_tweets_highvol = bitcoin_tweets_highvol[['user_name','date','text','hashtags','user_verified','user_followers'
                                          ,'user_friends','user_favourites']]


In [ ]:
bitcoin_tweets_highvol.head(5)

,user_name,date,text,hashtags,user_verified,user_followers,user_friends,user_favourites
0,DeSota Wilson,2021-02-10,Blue Ridge Bank shares halted by NYSE after #b...,['bitcoin'],False,8534.0,7605.0,4838.0
1,CryptoND,2021-02-10,"😎 Today, that's this #Thursday, we will do a ""...","['Thursday', 'Btc', 'wallet', 'security']",False,6769.0,1532.0,25483.0
2,Tdlmatias,2021-02-10,"Guys evening, I have read this article about B...",NaN,False,128.0,332.0,924.0
3,Crypto is the future,2021-02-10,$BTC A big chance in a billion! Price: \487264...,"['Bitcoin', 'FX', 'BTC', 'crypto']",False,625.0,129.0,14.0
4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,2021-02-10,This network is secured by 9 508 nodes as of t...,['BTC'],False,1249.0,1472.0,10482.0


In [ ]:
#Filtering the user_verified accounts and storing only those tweets for authentic tweets
bitcoin_tweets_highvol = bitcoin_tweets_highvol[bitcoin_tweets_highvol['user_verified']== True]
bitcoin_tweets_highvol = bitcoin_tweets_highvol.reset_index(drop = True)

In [ ]:
print(bitcoin_tweets_highvol['user_followers'].median())
print(bitcoin_tweets_highvol['user_followers'].mean())
print(bitcoin_tweets_highvol['user_followers'].mode())

100186.0
440543.00973897113
0    45762.0
Name: user_followers, dtype: float64


In [ ]:
print(bitcoin_tweets_highvol['user_favourites'].median())
print(bitcoin_tweets_highvol['user_favourites'].mean())
print(bitcoin_tweets_highvol['user_favourites'].mode())

8961.0
21871.401740192337
0    2963.0
Name: user_favourites, dtype: float64


In [ ]:
bitcoin_tweets_filtered = bitcoin_tweets_highvol.loc[(bitcoin_tweets_highvol['user_followers']>=100186.0) &
                                                  (bitcoin_tweets_highvol['user_favourites']>=8961.0)]
bitcoin_tweets_filtered = bitcoin_tweets_filtered.reset_index(drop = True)
bitcoin_tweets_filtered.shape

(10016, 8)

In [ ]:
import re

def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)        
    return input_txt

def clean_tweets(tweets_bitcoin):
    # remove twitter Return handles (RT @xxx:)
    tweets_bitcoin = [remove_pattern(tweet, "RT @[\w]*:") for tweet in tweets_bitcoin]
    
    # remove twitter handles (@xxx)
    tweets_bitcoin = [remove_pattern(tweet, "@[\w]*") for tweet in tweets_bitcoin]
    
    # remove URL links (httpxxx)
    tweets_bitcoin = [remove_pattern(tweet, "https?://[A-Za-z0-9./]*") for tweet in tweets_bitcoin]
    
    # remove special characters, numbers, punctuations (except for #)
    tweets_bitcoin = [re.sub("[^a-zA-Z0-9]", " ", tweet) for tweet in tweets_bitcoin]
    
    return tweets_bitcoin


In [ ]:
bitcoin_tweets_filtered['text'] = clean_tweets(bitcoin_tweets_filtered['text'])

In [ ]:
bitcoin_tweets_filtered.reset_index(inplace=True, drop=True)

In [ ]:
bitcoin_tweets_filtered.head(5)

,user_name,date,text,hashtags,user_verified,user_followers,user_friends,user_favourites
0,Blockstream,2021-02-10,BlockstreamAQUA is our new Bitcoin and wall...,"['BlockstreamAQUA', 'Bitcoin']",True,131970.0,794.0,12454.0
1,Keith McCullough,2021-02-09,Hedgeye Crypto Quant TC 2 7 ETH 2 8 ...,"['Bitcoin', 'BTC']",True,195162.0,745.0,15986.0
2,Blockstream,2021-02-09,Drop some sats pick up some Bitcoin stickers ...,"['Bitcoin', 'unfairlycheap']",True,131972.0,794.0,12456.0
3,TheStreet,2021-02-09,Bitcoin surges past 48 000 pulling other cry...,NaN,True,758587.0,1125.0,9277.0
4,TheStreet,2021-02-09,Stock futures are lower following the S amp P ...,NaN,True,758587.0,1125.0,9277.0


In [ ]:
#Converting the tweets to lowercase to maintain uniformity of data and removing extra whitespaces
bitcoin_tweets_filtered['text'] = bitcoin_tweets_filtered['text'].str.lower()
bitcoin_tweets_filtered['text'] = bitcoin_tweets_filtered['text'].str.lstrip()

In [ ]:
# after cleaning the tweets and using regex to retrieve only elon musk's tweets, we filtered our list from 24172 to 135 tweets
# importing libraries for sentiment analysis

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [ ]:
#Finding Polarity scores for each tweet
scores = []
# Declare variables for scores
compound_list = []
positive_list = []
negative_list = []
neutral_list = []
for i in range(bitcoin_tweets_filtered['text'].shape[0]):
#print(analyser.polarity_scores(sentiments_pd['text'][i]))
    compound = analyzer.polarity_scores(bitcoin_tweets_filtered['text'][i])["compound"]
    pos = analyzer.polarity_scores(bitcoin_tweets_filtered['text'][i])["pos"]
    neu = analyzer.polarity_scores(bitcoin_tweets_filtered['text'][i])["neu"]
    neg = analyzer.polarity_scores(bitcoin_tweets_filtered['text'][i])["neg"]
    
    scores.append({"Compound": compound,
                       "Positive": pos,
                       "Negative": neg,
                       "Neutral": neu
                  })

In [ ]:
#Storing the scores in a dataframe
score_sentiment = pd.DataFrame.from_dict(scores)

In [ ]:
score_sentiment.head(5)

,Compound,Positive,Negative,Neutral
0,0.4404,0.139,0.000,0.861
1,0.5106,0.191,0.000,0.809
2,-0.3612,0.000,0.163,0.837
3,0.0000,0.000,0.000,1.000
4,0.2732,0.156,0.080,0.764


In [ ]:
#joining the sentiment scores to the dataframe
bitcoin_tweets_filtered =  bitcoin_tweets_filtered.join(score_sentiment)

In [ ]:
bitcoin_tweets_filtered.tail(5)

,user_name,date,text,hashtags,user_verified,user_followers,user_friends,user_favourites,Compound,Positive,Negative,Neutral
10011,Altcoin Daily,2023-01-07,crypto news today huobi to layoff 20 of...,"['Huobi', 'BTC', 'ETH', 'TRON']",True,1320968.0,672.0,57367.0,0.5778,0.154,0.0,0.846
10012,Real Vision,2023-01-06,market conditions today dji 2 12 33 630...,"['DJI', 'SP500', 'NASDAQ', 'US10Y', 'DXY', 'WT...",True,362223.0,1825.0,23862.0,0.0000,0.000,0.0,1.000
10013,MANDO CT,2023-01-06,i smell bulls btc,['BTC'],True,100887.0,1697.0,58534.0,0.0000,0.000,0.0,1.000
10014,Lucas Nahuel Velasco,2023-01-06,shill me your best gem x100 btc eth doge...,"['BTC', 'ETH', 'DOGE', 'SHIB', 'cryptocurrency...",True,247412.0,4798.0,21964.0,0.6369,0.208,0.0,0.792
10015,"Adrian Zduńczyk, CMT",2023-01-06,seasonality in 2023 btc crypto amp stoc...,"['BTC', 'Crypto', 'Stocks']",True,656325.0,1587.0,95754.0,0.0000,0.000,0.0,1.000


In [ ]:
bitcoin_tweets_filtered.shape

(10016, 12)

In [ ]:
bitdata.head(5)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-07-04,9084.233398,9183.295898,9053.629883,9132.488281,12290528515,0.0,0.0
2020-07-05,9126.090820,9162.183594,8977.015625,9073.942383,12903406143,0.0,0.0
2020-07-06,9072.849609,9375.474609,9058.664062,9375.474609,17889263252,0.0,0.0
2020-07-07,9349.161133,9360.617188,9201.815430,9252.277344,13839652595,0.0,0.0
2020-07-08,9253.020508,9450.335938,9249.500000,9428.333008,19702359883,0.0,0.0


In [ ]:
#converting date into datetime format
bitdata.reset_index(level=0, drop = True)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,9084.233398,9183.295898,9053.629883,9132.488281,12290528515,0.0,0.0
1,9126.090820,9162.183594,8977.015625,9073.942383,12903406143,0.0,0.0
2,9072.849609,9375.474609,9058.664062,9375.474609,17889263252,0.0,0.0
3,9349.161133,9360.617188,9201.815430,9252.277344,13839652595,0.0,0.0
4,9253.020508,9450.335938,9249.500000,9428.333008,19702359883,0.0,0.0
...,...,...,...,...,...,...,...
1090,30086.187500,30796.250000,30057.203125,30445.351562,13180860821,0.0,0.0
1091,30441.353516,31256.863281,29600.275391,30477.251953,26387306197,0.0,0.0
1092,30471.847656,30641.289062,30328.865234,30590.078125,9086606733,0.0,0.0
1093,30587.269531,30766.140625,30264.019531,30620.769531,10533418042,0.0,0.0


In [ ]:
bitdata.reset_index(inplace=True)

In [ ]:
#Renaming the Date column to match the date column in the tweets dataframe
bitdata.rename(columns={"Date": "date"}, inplace = True)

In [ ]:
#converting date into datetime format
bitdata[['date']] = bitdata[['date']].apply(pd.to_datetime)
bitcoin_tweets_filtered['date'] = pd.to_datetime(bitcoin_tweets_filtered['date'])

In [ ]:
#Merging bitcoin tweets and historical data into one dataframe
bitcoin_tweets_filtered = bitcoin_tweets_filtered.merge(bitdata, on='date', how='left')

In [ ]:
bitcoin_tweets_filtered

,user_name,date,text,hashtags,user_verified,user_followers,user_friends,user_favourites,Compound,Positive,Negative,Neutral,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,Blockstream,2021-02-10,blockstreamaqua is our new bitcoin and walle...,"['BlockstreamAQUA', 'Bitcoin']",True,131970.0,794.0,12454.0,0.4404,0.139,0.000,0.861,46469.761719,47145.566406,43881.152344,44918.183594,87301089896,0.0,0.0
1,Keith McCullough,2021-02-09,hedgeye crypto quant tc 2 7 eth 2 8 ...,"['Bitcoin', 'BTC']",True,195162.0,745.0,15986.0,0.5106,0.191,0.000,0.809,46184.992188,48003.722656,45166.960938,46481.105469,91809846886,0.0,0.0
2,Blockstream,2021-02-09,drop some sats pick up some bitcoin stickers ...,"['Bitcoin', 'unfairlycheap']",True,131972.0,794.0,12456.0,-0.3612,0.000,0.163,0.837,46184.992188,48003.722656,45166.960938,46481.105469,91809846886,0.0,0.0
3,TheStreet,2021-02-09,bitcoin surges past 48 000 pulling other cry...,NaN,True,758587.0,1125.0,9277.0,0.0000,0.000,0.000,1.000,46184.992188,48003.722656,45166.960938,46481.105469,91809846886,0.0,0.0
4,TheStreet,2021-02-09,stock futures are lower following the s amp p ...,NaN,True,758587.0,1125.0,9277.0,0.2732,0.156,0.080,0.764,46184.992188,48003.722656,45166.960938,46481.105469,91809846886,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10011,Altcoin Daily,2023-01-07,crypto news today huobi to layoff 20 of...,"['Huobi', 'BTC', 'ETH', 'TRON']",True,1320968.0,672.0,57367.0,0.5778,0.154,0.000,0.846,16952.117188,16975.017578,16914.191406,16955.078125,7714767174,0.0,0.0
10012,Real Vision,2023-01-06,market conditions today dji 2 12 33 630...,"['DJI', 'SP500', 'NASDAQ', 'US10Y', 'DXY', 'WT...",True,362223.0,1825.0,23862.0,0.0000,0.000,0.000,1.000,16836.472656,16991.994141,16716.421875,16951.968750,14413662913,0.0,0.0
10013,MANDO CT,2023-01-06,i smell bulls btc,['BTC'],True,100887.0,1697.0,58534.0,0.0000,0.000,0.000,1.000,16836.472656,16991.994141,16716.421875,16951.968750,14413662913,0.0,0.0
10014,Lucas Nahuel Velasco,2023-01-06,shill me your best gem x100 btc eth doge...,"['BTC', 'ETH', 'DOGE', 'SHIB', 'cryptocurrency...",True,247412.0,4798.0,21964.0,0.6369,0.208,0.000,0.792,16836.472656,16991.994141,16716.421875,16951.968750,14413662913,0.0,0.0


In [ ]:
#Dropping the columns that are not required for the analysis
bitcoin_tweets_filtered = bitcoin_tweets_filtered.drop(['Dividends', 'Stock Splits'], axis = 1)

In [ ]:
bitcoin_tweets_filtered.head(5)

,user_name,date,text,hashtags,user_verified,user_followers,user_friends,user_favourites,Compound,Positive,Negative,Neutral,Open,High,Low,Close,Volume
0,Blockstream,2021-02-10,blockstreamaqua is our new bitcoin and walle...,"['BlockstreamAQUA', 'Bitcoin']",True,131970.0,794.0,12454.0,0.4404,0.139,0.000,0.861,46469.761719,47145.566406,43881.152344,44918.183594,87301089896
1,Keith McCullough,2021-02-09,hedgeye crypto quant tc 2 7 eth 2 8 ...,"['Bitcoin', 'BTC']",True,195162.0,745.0,15986.0,0.5106,0.191,0.000,0.809,46184.992188,48003.722656,45166.960938,46481.105469,91809846886
2,Blockstream,2021-02-09,drop some sats pick up some bitcoin stickers ...,"['Bitcoin', 'unfairlycheap']",True,131972.0,794.0,12456.0,-0.3612,0.000,0.163,0.837,46184.992188,48003.722656,45166.960938,46481.105469,91809846886
3,TheStreet,2021-02-09,bitcoin surges past 48 000 pulling other cry...,NaN,True,758587.0,1125.0,9277.0,0.0000,0.000,0.000,1.000,46184.992188,48003.722656,45166.960938,46481.105469,91809846886
4,TheStreet,2021-02-09,stock futures are lower following the s amp p ...,NaN,True,758587.0,1125.0,9277.0,0.2732,0.156,0.080,0.764,46184.992188,48003.722656,45166.960938,46481.105469,91809846886


In [ ]:
#checking for null valuesn in the dataframe
bitcoin_tweets_filtered.isnull().sum()

user_name           0
date                0
text                0
hashtags           17
user_verified       0
user_followers      0
user_friends        0
user_favourites     0
Compound            0
Positive            0
Negative            0
Neutral             0
Open                0
High                0
Low                 0
Close               0
Volume              0
dtype: int64

In [ ]:
bitcoin_tweets_filtered.columns

Index(['user_name', 'date', 'text', 'hashtags', 'user_verified',
       'user_followers', 'user_friends', 'user_favourites', 'Compound',
       'Positive', 'Negative', 'Neutral', 'Open', 'High', 'Low', 'Close',
       'Volume'],
      dtype='object')

In [ ]:
bitcoin_tweets_filtered.head(5)

,user_name,date,text,hashtags,user_verified,user_followers,user_friends,user_favourites,Compound,Positive,Negative,Neutral,Open,High,Low,Close,Volume
0,Blockstream,2021-02-10,blockstreamaqua is our new bitcoin and walle...,"['BlockstreamAQUA', 'Bitcoin']",True,131970.0,794.0,12454.0,0.4404,0.139,0.000,0.861,46469.761719,47145.566406,43881.152344,44918.183594,87301089896
1,Keith McCullough,2021-02-09,hedgeye crypto quant tc 2 7 eth 2 8 ...,"['Bitcoin', 'BTC']",True,195162.0,745.0,15986.0,0.5106,0.191,0.000,0.809,46184.992188,48003.722656,45166.960938,46481.105469,91809846886
2,Blockstream,2021-02-09,drop some sats pick up some bitcoin stickers ...,"['Bitcoin', 'unfairlycheap']",True,131972.0,794.0,12456.0,-0.3612,0.000,0.163,0.837,46184.992188,48003.722656,45166.960938,46481.105469,91809846886
3,TheStreet,2021-02-09,bitcoin surges past 48 000 pulling other cry...,NaN,True,758587.0,1125.0,9277.0,0.0000,0.000,0.000,1.000,46184.992188,48003.722656,45166.960938,46481.105469,91809846886
4,TheStreet,2021-02-09,stock futures are lower following the s amp p ...,NaN,True,758587.0,1125.0,9277.0,0.2732,0.156,0.080,0.764,46184.992188,48003.722656,45166.960938,46481.105469,91809846886


In [ ]:
bitcoin_tweets_filtered = bitcoin_tweets_filtered[[ 'date', 'text',
       'user_followers', 'user_favourites', 'Compound',
        'Open','High','Low', 'Close',
        'Volume']]


In [ ]:
bitcoin_tweets_filtered.tail()

,date,text,user_followers,user_favourites,Compound,Open,High,Low,Close,Volume
10011,2023-01-07,crypto news today huobi to layoff 20 of...,1320968.0,57367.0,0.5778,16952.117188,16975.017578,16914.191406,16955.078125,7714767174
10012,2023-01-06,market conditions today dji 2 12 33 630...,362223.0,23862.0,0.0000,16836.472656,16991.994141,16716.421875,16951.968750,14413662913
10013,2023-01-06,i smell bulls btc,100887.0,58534.0,0.0000,16836.472656,16991.994141,16716.421875,16951.968750,14413662913
10014,2023-01-06,shill me your best gem x100 btc eth doge...,247412.0,21964.0,0.6369,16836.472656,16991.994141,16716.421875,16951.968750,14413662913
10015,2023-01-06,seasonality in 2023 btc crypto amp stoc...,656325.0,95754.0,0.0000,16836.472656,16991.994141,16716.421875,16951.968750,14413662913


In [ ]:
#checking for null values in the dataframe
bitcoin_tweets_filtered.isnull().sum()

date               0
text               0
user_followers     0
user_favourites    0
Compound           0
Open               0
High               0
Low                0
Close              0
Volume             0
dtype: int64

In [ ]:
#converting the dataframe to csv file and saving it
bitcoin_tweets_filtered.to_csv('Bitcoin_tweets_Price_data.csv')
